<a href="https://colab.research.google.com/github/anna-alt/AI-Lab/blob/main/SceneRecognition1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, SubsetRandomSampler, ConcatDataset
from torchvision import datasets, transforms
from torchvision import models

import numpy as np
import sklearn



In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

device

device(type='cuda')

In [13]:
trainset_size = 5000
val_size = 500
num_epochs = 10
image_size = (224,224)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Resize(image_size),
                              
                              ])
# Download and load the training data
trainset = datasets.SUN397('/content/drive/MyDrive/Data', download= False, transform=transform)
splittrain = torch.utils.data.random_split(trainset, [trainset_size, len(trainset) - trainset_size])[0]
train_loader = DataLoader(splittrain, batch_size=25,num_workers = 2, shuffle=True)

validationset = datasets.SUN397("/content/drive/MyDrive/Data", download=False, transform=transform)
splitvalidation = torch.utils.data.random_split(validationset, [val_size, len(trainset) - val_size])[0]
val_loader = DataLoader(splitvalidation, batch_size=25,num_workers = 2, shuffle=True)



In [ ]:
for image, label in train_loader:
  labels = np.array(image)
  numpy_labels = label.numpy()
  print(labels)
  break


AttributeError: ignored

In [ ]:
model = models.resnet50(weights = "DEFAULT")
model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=6e-5)
criterion = nn.CrossEntropyLoss()
print(model)

In [16]:
def train(epoch, log_interval=20):
    # Set model to training mode
    model.train()
    
    # Loop over each batch from the training set
    for batch_idx, (data, target) in enumerate(train_loader):
        # Copy data to GPU if needed
        data = data.to(device)
        target = target.to(device)

        # Zero gradient buffers
        optimizer.zero_grad() 
        
        # Pass data through the network
        output = model(data)

        # Calculate loss
        loss = criterion(output, target)

        # Backpropagate
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

In [11]:

def validate(loss_vector, accuracy_vector):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in val_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    val_loss /= len(val_loader)
    loss_vector.append(val_loss)

    print(pred)

    accuracy = 100. * correct.to(torch.float32) / len(val_loader.dataset)
    accuracy_vector.append(accuracy)

    
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(val_loader.dataset), accuracy))

In [22]:
def Predic(loss_vector,accuracy_vector):
    val_loss, correct = 0, 0
    for data, target in val_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
    print( pred)
    print("/n")
    print(correct)


In [23]:
lossv, accv = [], []
for epoch in range(1, num_epochs + 1):
    train(epoch)
    validate(lossv, accv)
    Predic(lossv,accv)

Train Epoch: 1 [0/5000 (0%)]	Loss: 0.012440
Train Epoch: 1 [500/5000 (10%)]	Loss: 0.007685
Train Epoch: 1 [1000/5000 (20%)]	Loss: 0.029098
Train Epoch: 1 [1500/5000 (30%)]	Loss: 0.010422
Train Epoch: 1 [2000/5000 (40%)]	Loss: 0.054140
Train Epoch: 1 [2500/5000 (50%)]	Loss: 0.033673
Train Epoch: 1 [3000/5000 (60%)]	Loss: 0.008233
Train Epoch: 1 [3500/5000 (70%)]	Loss: 0.014845
Train Epoch: 1 [4000/5000 (80%)]	Loss: 0.009939
Train Epoch: 1 [4500/5000 (90%)]	Loss: 0.017391

Validation set: Average loss: 2.7626, Accuracy: 214/500 (43%)

tensor([158, 227, 227, 259, 107,  14, 213, 127, 263,  47, 115, 279, 270,  72,
        133, 234, 359, 336, 358, 331, 327, 342, 228,  43,  43],
       device='cuda:0')
/n
tensor(214)
Train Epoch: 2 [0/5000 (0%)]	Loss: 0.005537
Train Epoch: 2 [500/5000 (10%)]	Loss: 0.006034
Train Epoch: 2 [1000/5000 (20%)]	Loss: 0.010377
Train Epoch: 2 [1500/5000 (30%)]	Loss: 0.011989
Train Epoch: 2 [2000/5000 (40%)]	Loss: 0.003165
Train Epoch: 2 [2500/5000 (50%)]	Loss: 0.0048

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))